In [147]:
%load_ext autoreload
%autoreload 2

import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.stats import *
from utils.performance import *
from plotly.subplots import make_subplots
from account import Binance
from binance.client import Client
from pandas.core.frame import DataFrame
import pandas as pd
import warnings
from strategy_v3.Strategy import *
from strategy_v3.Executor import ExecutorBinance, ExecutorBacktest
from strategy_v3.DataLoader import DataLoaderBinance
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
from time import sleep
from tqdm import tqdm


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
symbol = 'BTCFDUSD'
binance = Binance()
df = binance.get_historical_instrument_price(symbol, interval='1m', start_str='1 Day ago')
plot_price_ohcl(df, symbol)

# Grid Trading Logic (Arithmetic)

### For each time interval, iterate thru below steps

1. check if status is idle (i.e. no outstanding grid orders) and hurst exponent to see if this indicates mean-reverting trends

2. If both yes for above, place grid orders via LIMIT ORDER

- num orders   = grid_size * 2
- grid spacing = historical volatility * vol scale
- stop loss    = historical volatility * vol scale * vol_stoploss_scale

3. fill the orders using high and low (backtest mode only)

4. check if status neutral (i.e. have filled grid orders but positions are neutral). If yes, cancel all orders

5. check if current price triggers stop-loss. If yes, cancel all orders and close out position via MARKET ORDER.

### Notes

Improvement: we should be able to predict either mean-reverting or trending and place a sutiable grids

e.g. if the price is trending up, instead of placing a grid of 5-sells and 5-buys centered at current price, we can place a grid with 5-sells and 5-buys centered at current price + 1% to capture the momentum


## Calibration logging

2024-02-27: Attempt to use rolling average metrics based on looking back 2 * half life interval. Replace vol with half-life vol and center price from spot to half-life SMA close

-  Tested BTCFDUSD on 15days. original cum returns is 6% whereas new change is 3% only

2024-02-29: Attempt to use momentum order when hurst exponent is >= 0.6 and use extra momentum filters (Spot > T-5 > T-10) to put the momentum grid orders.

-  Tested BTCFDUSD. this added more return on original strategy, because this is mutually exclusive with mean reverting orders, this enhance return during non-mean-reverting periods 

## BackTest Strategy

In [15]:
def backtest_strategy(strategy, start_str:str):
    strategy.load_data(start_str)    
    df = strategy.df.copy()
    with tqdm(total = len(df)) as pbar:
        for _, data in df.iterrows():
            strategy.execute(data)
            pbar.update(1)
    strategy.close_out_positions('close', data['Close'], data['Date'])
    return strategy

In [148]:
strategy1 = GridArithmeticStrategy(
    instrument = 'BTCFDUSD',    
    interval = '5m',
    grid_size = 5,
    vol_lookback = 30,
    vol_grid_scale = 0.2,
    vol_stoploss_scale = 7,
    position_size = 300,
    hurst_exp_mr_threshold = 0.5,
    hurst_exp_mo_threshold = 0.6,
    verbose=False,
)
strategy1.set_price_decimal(2)
strategy1.set_qty_decimal(5)
strategy1.set_data_loder(DataLoaderBinance())
strategy1.set_executor(ExecutorBacktest())

In [149]:
strategy1 = backtest_strategy(strategy1, '5 Days Ago')

100%|██████████| 1340/1340 [00:19<00:00, 69.98it/s]


In [150]:
strategy1.summary()

,Measure,grid_28677
0,Cumulative Return,1.038894
1,Annualized Return,2.955003
2,Annualized Volatility,0.072992
3,Annualized Sharpe Ratio,39.792462
4,Maximum Drawdown,-0.003184


# Execute Strategy

Strategy ID - all orders are marked by strategy id

Therefore, all orders / pnl logic are based on same set of orders which isolates from other strategy or previous strategy

If we want to continue previous strategy (either manual exit or error exit), we need to put the same strategy id. so the pnl could be accumlated from last time.

<b>Notes: execute.py runs the actual strategy, below code is just to keep track the strategy status and performance</b>

### Strategy logging:

-   903492 - traded on BTCUSDT for 1.5days. gross pnl is good, but realized that there is 0.1% transaction cost which overwhelme all the PNL

-   v1 - traded on BTCFDUSD - no transaction cost. 
    -  Corrected the definition of price volatility from close.diff.std to close.std and change the vol_grid_scale from 0.4 to 0.2.
    -  Discover that the PnL seems to increase with smaller grid spacing
    -  Start using momentum orders

In [151]:
strategy = GridArithmeticStrategy(
    instrument = 'BTCFDUSD',        
    interval = '5m',
    grid_size = 5,
    vol_lookback = 30,
    vol_grid_scale = 0.4,
    vol_stoploss_scale = 7,
    position_size = 100,
    hurst_exp_mr_threshold = 0.5,
    hurst_exp_mo_threshold = 0.6,
)

strategy.set_price_decimal(2)
strategy.set_qty_decimal(5)
strategy.set_data_loder(DataLoaderBinance())
strategy.set_executor(ExecutorBinance())
strategy.set_strategy_id('v1')
strategy.load_data('2024-02-25')
strategy.summary(True)

,Measure,grid_v1
0,Cumulative Return,1.014028
1,Annualized Return,2.065975
2,Annualized Volatility,0.130144
3,Annualized Sharpe Ratio,15.486707
4,Maximum Drawdown,-0.006294
